# Import

In [2]:
import numpy as np
import cv2 as cv
import pickle
import time
import random
random.seed(50)

# Initiator

In [3]:
def rgb_to_hex(r, g, b):
    return ('{:X}{:X}{:X}').format(r, g, b)

In [4]:
def rgb_to_hex_image(image):
    image = image.astype(object)
    x,y,*_ = image.shape
    for i in range(x):
        for j in range(y):
            r,g,b = image[i,j]
#             x = rgb_to_hex_image(int(r),int(g),int(b))
#             x = ('{:X}{:X}{:X}').format(int(r),int(g),int(b))
            image[i,j,0] ="{:024b}".format(int(('{:X}{:X}{:X}').format(int(r),int(g),int(b)),16))
    return image[:,:,0]

In [5]:
def pixelmatrix(matrix, pixel=24):
    shape = list(matrix.shape)
    for x in range(len(shape)):
        shape[x] = int(np.ceil(shape[x] / 10) * 10)
    shape = shape+list([24])
    output = np.zeros(shape)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            output[i,j,:] = list(matrix[i,j])
    return output

In [6]:
def anchorcreater(matrix, anchorshape=[10, 10]):
    shape = anchorshape + list(matrix.shape)
    x, y = [int(z / 10) for z in shape[2:4]]
    for index in range(2, 4):
        shape[index] = int(shape[index] / 10)
    output = np.zeros(shape)
    ymin, ymax = 0, 10
    for yanchor in range(y):
        xmin = 0
        xmax = 10
        for xanchor in range(x):
            output[:, :, xanchor, yanchor, :] = matrix[xmin:xmax, ymin:ymax, :]
            xmin += 10
            xmax += 10
        ymin += 10
        ymax += 10
    return output

In [7]:
def last_matrix(image):
    return anchorcreater(pixelmatrix(rgb_to_hex_image(image)))

# Mediator

take 10x10 matrix

In [7]:
def solution(matrix,filled=True):
    condition = 1 if filled else 0
    rows, columns = np.where(matrix == condition)
    output = []
    for index in range(len(rows)):
        output.append([rows[index], columns[index]])
    return output

In [8]:
def hv_read(matrix):
    height, width = matrix.shape
    horizontal_read = [[[matrix[j, i]] for i in range(width)] for j in range(height)]
    vertical_read = [[[matrix[j, i]] for j in range(height)] for i in range(width)]
    horizontal_clues = list(map(_nono_creator_, horizontal_read))
    vertical_clues = list(map(_nono_creator_, vertical_read))

    return horizontal_clues, vertical_clues

In [9]:
def _nono_creator_(inp):
    arr = inp
    i = 0
    clues = []
    count = 0
    if len(arr) == 0:
        return []
    curr = arr[0]
    while i < len(arr):
        while i < len(arr) and str(arr[i]) == str(curr):
            i += 1
            count += 1
        clues.append((curr, count))
        if i < len(arr):
            count = 0
            curr = arr[i]
    output = []
    for element in clues:
        if element[0][0] == 1:
            output.append(element[1])
    return output

In [10]:
class Mediator(object):
    def __init__(self, matrix):
        self.n_rows, self.n_cols = matrix.shape
        self.rows_constraints, self.cols_constraints = hv_read(matrix)
        self.solution_list = solution(matrix)

# Solver

take Mediator object of 10x10 matrix

In [11]:
def possibilities_generator(
        prior, min_pos, max_start_pos, constraint_len, total_filled):

    prior_filled = np.zeros(len(prior)).astype(bool)
    prior_filled[prior == 1] = True
    prior_empty = np.zeros(len(prior)).astype(bool)
    prior_empty[prior == 0] = True
    for start_pos in range(min_pos, max_start_pos + 1):
        possible = -1 * np.ones(len(prior))
        possible[start_pos:start_pos + constraint_len] = 1
        if start_pos + constraint_len < len(possible):
            possible[start_pos + constraint_len] = 0
        if start_pos > 0:
            possible[start_pos - 1] = 0

        # add in the prior
        possible[np.logical_and(possible == -1, prior == 0)] = 0
        possible[np.logical_and(possible == -1, prior == 1)] = 1

        # if contradiction with prior, continue
        # 1. possible changes prior = 1 to something else
        # 2. possible changes prior = 0 to something else
        # 3. everything is assigned in possible but there are not
        #    enough filled in
        # 4. possible changes nothing about the prior
        if np.any(possible[np.where(prior == 1)[0]] != 1) or \
                np.any(possible[np.where(prior == 0)[0]] != 0) or \
                np.sum(possible == 1) > total_filled or \
                (np.all(possible >= 0) and np.sum(possible == 1) <
                 total_filled) or \
                np.all(prior == possible):
            continue
        yield possible

In [28]:
def generate_constraints_line(constraints, prior, size):
    """
    example:
    row length = 4, row constraint = 2 (it is the only constraint)
    min_pos = 0, max_pos = 3, max_start_pos = 2
    row length = 5, row constraints 1, 2
    constraint 1:
        min_pos = 0, max_pos = 1, max_start_pos = 1
    constraint 2:
        min_pos = 2, max_pos = 4, max_start_pos = 3
    """
    if len(constraints) == 0:
        return np.logical_not(np.ones(len(prior))), np.logical_not(np.zeros(len(prior)))

    total_filled = np.sum(constraints)

    # if the number of filled in squares is equal to the total
    # constraints, fill in the rest with empties and call it a day
    if np.sum(prior == 1) == total_filled:
        for index in range(len(prior)):
            if prior[index] == -1:
                prior[index] = 0
        return (prior == 1, prior == 0)

    # if there is only one square left un-solved, solve it
    index_unknown = np.where(prior == -1)[0]
    if len(index_unknown) == 1:
        currently_filled_in_prior = np.sum(prior == 1)
        if total_filled == currently_filled_in_prior:
            prior[index_unknown[0]] = 0
        else:
            prior[index_unknown[0]] = 1

        return (prior == 1, prior == 0)

    # if any constraints are solved, don't
    # iterate over them
    indices_to_remove = []
    constraint_indices = np.ones(len(constraints))

    start_blocks = []
    start_would_be_zero_index = []
    block = 0
    for index, item in zip(list(range(len(prior))), prior):
        if item != -1 and item == 1:
            block += 1
        else:
            if block > 0:
                start_would_be_zero_index.append(index)
                start_blocks.append(block)
                block = 0
            if item == -1:
                break
    print(start_blocks)

    index = 0
    for block, constraint in zip(
            start_blocks, constraints[:len(start_blocks)]):
        
        print(constraints[:len(start_blocks)],len(start_blocks),constraints)
        if block == constraint:
            # remove from iteration
            indices_to_remove.append(index)
            # if the block is correct, update the prior appropriately
            prior[start_would_be_zero_index[index]] = 0
            before_index = start_would_be_zero_index[index] - constraint - 1
            if before_index > 0:
                prior[before_index] = 0
        else:
            break
        index += 1

    end_blocks = []
    end_would_be_zero_index = []
    reverse_index_range = list(range(len(prior)))
    reverse_index_range = reversed(reverse_index_range)
    block = 0
    for index, item in zip(reverse_index_range, prior[::-1]):
        if item != -1 and item == 1:
            block += 1
        else:
            if block > 0:
                end_would_be_zero_index.append(index)
                end_blocks.append(block)
                block = 0
            if item == -1:
                break
    if block > 0:
        end_would_be_zero_index.append(index)
        end_blocks.append(block)

    index = len(constraints) - 1
    for block, constraint in zip(
            end_blocks[::-1],
            constraints[-1 * len(end_blocks):]):
        if block == constraint:
            # remove from iteration
            indices_to_remove.append(index)
            # if the block is correct, update the prior appropriately
            prior[end_would_be_zero_index[len(constraints) - 1 - index]] = 0
            before_index = end_would_be_zero_index[
                               len(constraints) - 1 - index] + constraint + 1
            if before_index > 0 and before_index < len(prior):
                prior[before_index] = 0
        else:
            break
        index -= 1

    for index in indices_to_remove:
        constraint_indices[index] = 0

    # if all the constraints are met, make sure the
    # -1s are 0
    if np.all(constraint_indices == 0):
        for index in range(len(prior)):
            if prior[index] == -1:
                prior[index] = 0
        return (prior == 1, prior == 0)

    possibilities_filled = prior == 1
    possibilities_empty = prior == 0

    for constraint_index, constraint in zip(
            list(range(len(constraints))), constraints):
        if constraint_indices[constraint_index] == 0:
            continue

        possibilities_filled_temp = np.ones(size).astype(bool)
        possibilities_empty_temp = np.ones(size).astype(bool)
        min_pos = sum(constraints[:constraint_index]) + constraint_index
        max_pos = size - 1 - (
                sum(constraints[constraint_index + 1:]) +
                len(constraints[constraint_index + 1:]))

        # if we already have everything in that window
        # just continue
        if np.all(prior[min_pos:max_pos + 1] != -1):
            possibilities_filled[min_pos:max_pos + 1] = \
                prior[min_pos:max_pos + 1] == 1
            possibilities_empty[min_pos:max_pos + 1] = \
                prior[min_pos:max_pos + 1] == 0
            continue

        # if you have some contiguous thing from the prior in the range of
        # min_pos and max_pos, fix it
        potential_indices = np.where(prior[min_pos:max_pos + 1] != 0)[0]
        if len(potential_indices) > 0:
            potential_indices += min_pos
            min_pos = min(potential_indices)
            max_pos = max(potential_indices)

        max_start_pos = max_pos + 1 - constraint

        # the smart logic about the constraints is generated
        # in the possibilities_generator
        i = 0
        for possible_nums in possibilities_generator(
                prior, min_pos, max_start_pos, constraint, total_filled):
            possible_filled = possible_nums == 1
            possible_empty = possible_nums != 1
            possibilities_filled_temp = np.logical_and(
                possibilities_filled_temp, possible_filled)
            possibilities_empty_temp = np.logical_and(
                possibilities_empty_temp, possible_empty)
            i += 1

        if i > 0:
            possibilities_filled[min_pos:max_pos + 1] = \
                possibilities_filled_temp[min_pos:max_pos + 1]
            possibilities_empty[min_pos:max_pos + 1] = \
                possibilities_empty_temp[min_pos:max_pos + 1]

        # TODO: you can probably end early here. figure out the constraint
        # if np.all(possibilities_filled == False) and \
        #         np.all(possibilities_empty == False):
        #     break
    return (possibilities_filled, possibilities_empty)

In [13]:
def generate_solutions(
        n_rows, n_cols, rows_constraints, cols_constraints, puzzle):
    n_changed = 1
    iters = 0
    while n_changed > 0:
        n_changed = 0
        for row_index, row_constraints in enumerate(rows_constraints):
            # if the row has no empty spaces left,
            # just skip that set of constraints
            if np.sum(puzzle[row_index, :] == -1) == 0:
                continue

            (possibilities_filled, possibilities_empty) = \
                generate_constraints_line(
                    row_constraints,
                    puzzle[row_index, :].copy(),
                    n_cols)

            for index in range(len(possibilities_filled)):
                if possibilities_filled[index] and \
                        puzzle[row_index, index] == 1:
                    possibilities_filled[index] = False

            for index in range(len(possibilities_empty)):
                if possibilities_empty[index] and \
                        puzzle[row_index, index] == 0:
                    possibilities_empty[index] = False

            if np.all(np.logical_not(possibilities_filled)) and \
                    np.all(np.logical_not(possibilities_empty)):
                continue

            puzzle[row_index, :][possibilities_filled] = 1
            puzzle[row_index, :][possibilities_empty] = 0

            n_changed += sum(possibilities_filled)
            n_changed += sum(possibilities_empty)

#         print("n_changed = %s" % n_changed)

        # cols
        for col_index, col_constraints in zip(list(range(n_cols)), cols_constraints):
            # if the row has no empty spaces left,
            # just skip that set of constraints
            if np.sum(puzzle[:, col_index] == -1) == 0:
                continue

            (possibilities_filled, possibilities_empty) = \
                generate_constraints_line(
                    col_constraints,
                    puzzle[:, col_index].copy(),
                    n_rows)

            for index in range(len(possibilities_filled)):
                if possibilities_filled[index] and \
                        puzzle[index, col_index] == 1:
                    possibilities_filled[index] = False

            for index in range(len(possibilities_empty)):
                if possibilities_empty[index] and \
                        puzzle[index, col_index] == 0:
                    possibilities_empty[index] = False

            if np.all(np.logical_not(possibilities_filled)) and \
                    np.all(np.logical_not(possibilities_empty)):
                continue

            puzzle[:, col_index][possibilities_filled] = 1
            puzzle[:, col_index][possibilities_empty] = 0

            n_changed += sum(possibilities_filled)
            n_changed += sum(possibilities_empty)

#         print("n_changed = %s" % n_changed)
#         print("finished iter %s" % iters)

        iters += 1
    return puzzle

In [14]:
generate_constraints_line_no = 0 #876
possibilities_generator_no = 0 #1379
generate_solutions_no = 0 #17

In [15]:
class NonogramSolver(object):
    def __init__(self, nonogram):
        self.nonogram = nonogram
        self.puzzle_state = -1 * np.ones((nonogram.n_rows, nonogram.n_cols))
        self.filled_positions_hint_eligible = nonogram.solution_list
        self.prefilled_positions = []
        self.position_hints = {0:[],
                             1:[]}

    def _generate_solutions(self):
        self.puzzle_state = generate_solutions(
            self.nonogram.n_rows, self.nonogram.n_cols,
            self.nonogram.rows_constraints,
            self.nonogram.cols_constraints,
            self.puzzle_state)
        
    def _pick_help_square(self, position=None):
        empty_position = np.where(self.puzzle_state==-1)
        if len(empty_position)>0:
            empty_position = list(zip(empty_position[0],empty_position[1]))
        selected_position = random.choice(empty_position)
        filled_value = 1 if selected_position in self.filled_positions_hint_eligible else 0
        self.puzzle_state[selected_position]=filled_value
        self.position_hints[filled_value] = selected_position
    def _puzzle_is_solved(self):
        if np.sum(self.puzzle_state == -1) == 0:
            return True
        return False

# Pipeline

In [16]:
def pipeline(image):
    count = 0
    solvable_matrix = 0
    not_solvable_matrix = 0
    take_help = 0
    take_help_and_solved = 0
    start_time = time.time()
    matrix = last_matrix(image)
    x, y, z = matrix.shape[2:]
    z=1
    
    dictionary_output = {}
    for z_axis in range(z):
        dictionary_y = {}
        for y_axis in range(y):
            dictionary_x = {}
            for x_axis in range(x):
                count +=1
                filled_places = np.sum(matrix[:, :, x_axis, y_axis, z_axis])
                if 20 >= filled_places > 0:
                    output = solution(matrix[:, :, x_axis, y_axis, z_axis])
                    dictionary_x[x_axis]={"filled places":output}
                elif 80<=filled_places<100:
                    output = solution(matrix[:, :, x_axis, y_axis, z_axis],filled=False)
                    dictionary_x[x_axis]={"empty places":output}
                elif filled_places==100:
                    dictionary_x[x_axis]={"full filled":True}
                elif filled_places>20:
                    clues = Mediator(matrix[:, :, x_axis, y_axis, z_axis])
                    nonogram = NonogramSolver(clues)
                    nonogram._generate_solutions()
                    solvable = nonogram._puzzle_is_solved()
                    if solvable:
                        solvable_matrix += 1
                        dictionary_x[x_axis] = {"constraints": [clues.rows_constraints, clues.cols_constraints]}
                    else:
                        while not nonogram._puzzle_is_solved() and len(np.where(nonogram.puzzle_state==-1)[0])>0:
                            take_help += 1
                            nonogram._pick_help_square()
                            nonogram._generate_solutions()
                        if not nonogram._puzzle_is_solved():
                            not_solvable_matrix += 1
                        take_help_and_solved += 1
                        dictionary_x[x_axis] = {"constraints": [clues.rows_constraints, clues.cols_constraints],
                                                "positions": nonogram.position_hints}
#                 print("count=",count)
            if len(dictionary_x.keys()) > 0:
                dictionary_y[y_axis] = dictionary_x
        if len(dictionary_y.keys()) > 0:
            dictionary_output[z_axis] = dictionary_y
#         print("count=",count,"solvable_matrix",solvable_matrix+take_help_and_solved,"take_help",take_help,"not_solvable_matrix",not_solvable_matrix)
    return time.time()-start_time, dictionary_output

In [17]:
pic_name="picture.jpg"
img = cv.imread(pic_name)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
ttime, output = pipeline(img)
print(f'time: {ttime // 60}min {ttime % 60}sec')
print(constraints[:len(start_blocks)],len(start_blocks),constraints)

[7] 1 [7]
[7] 1 [7]
[6] 1 [6]
[6] 1 [6]
[6] 1 [6]
[6] 1 [6]
[2] 1 [2, 3]
[3] 1 [3, 3]
[2] 1 [2, 3]
[3] 1 [3, 4]
[3] 1 [3, 3]
[6] 1 [6]
[5] 1 [5]
[6] 1 [6]
[6] 1 [6]
[3] 1 [3, 4]
[3] 1 [3, 4]
[2] 1 [2, 3]
[8] 1 [8]
[6] 1 [6]
[4] 1 [4]
[4] 1 [4]
[4] 1 [4]
[2] 1 [2, 1]
[3] 1 [3]
[5] 1 [5]
[5] 1 [5]
[6] 1 [6, 2]
[10] 1 [10]
[10] 1 [10]
[1] 1 [1, 2, 2]
[1] 1 [1, 2]
[4] 1 [4, 2]
[1] 1 [1, 2, 2]
[1] 1 [1, 2]
[4] 1 [4, 2]
[1] 1 [1, 3]
[2] 1 [2, 2]
[1] 1 [1, 2, 2]
[1] 1 [1, 2, 4]
[2] 1 [2, 3]
[8] 1 [8]
[8] 1 [8]
[4] 1 [4, 3]
[4] 1 [4, 3]
[5] 1 [5]
[5] 1 [5]
[5] 1 [5]
[4] 1 [4]
[8] 1 [8, 1]
[8] 1 [8, 1]
[8] 1 [8, 1]
[8] 1 [8, 1]
[1] 1 [1, 2]
[2] 1 [2, 2]
[2] 1 [2, 3]
[2] 1 [2, 3]
[1] 1 [1, 3]
[1] 1 [1, 2]
[1] 1 [1, 2]
[1] 1 [1, 1]
[8] 1 [8]
[8] 1 [8]
[9] 1 [9]
[9] 1 [9]
[10] 1 [10]
[10] 1 [10]
[8] 1 [8, 1]
[8] 1 [8]
[7] 1 [7, 1]
[3] 1 [3, 2]
[2] 1 [2, 2]
[1] 1 [1, 2]
[4] 1 [4]
[4] 1 [4]
[5] 1 [5]
[5] 1 [5]
[2] 1 [2]
[3] 1 [3, 2]
[3] 1 [3, 2]
[9] 1 [9]
[9] 1 [9]
[9] 1 [9]
[8] 1 [8]
[7] 1 [7]
[4] 

In [18]:
len(output.keys())

1

In [19]:
output

{0: {0: {16: {'constraints': [[[],
      [],
      [],
      [2],
      [6],
      [8],
      [7],
      [10],
      [10],
      [10]],
     [[3], [3], [1, 3], [5], [6], [6], [6], [6], [7], [7]]]},
   17: {'full filled': True},
   18: {'constraints': [[[10], [10], [10], [10], [10], [9], [1], [], [], []],
     [[7], [6], [6], [6], [6], [6], [6], [6], [6], [5]]]}},
  1: {16: {'constraints': [[[],
      [],
      [4, 1],
      [10],
      [10],
      [10],
      [10],
      [10],
      [10],
      [10]],
     [[8], [8], [8], [8], [7], [7], [8], [7], [7], [7]]]},
   17: {'full filled': True},
   18: {'constraints': [[[10], [10], [10], [10], [8], [], [], [], [], []],
     [[5], [5], [5], [5], [5], [5], [5], [5], [4], [4]]]}},
  2: {16: {'empty places': [[0, 0],
     [0, 1],
     [0, 2],
     [0, 3],
     [0, 4],
     [0, 5],
     [0, 6],
     [0, 7],
     [0, 8],
     [0, 9],
     [1, 0],
     [1, 1],
     [1, 2],
     [1, 3],
     [1, 4],
     [1, 5],
     [1, 8],
     [1, 9]]},
   17: {'f

In [29]:
con = np.array([3,4])
prior = -1 * np.ones(10)
prior[:3] = 1
prior[3:5]=0
prior[5:8]=1
prior
size = 10

In [30]:
generate_constraints_line(con,prior,size)
print(constraints[:len(start_blocks)],len(start_blocks),constraints)

[3, 3]
[3 4] 2 [3 4]
[3 4] 2 [3 4]


(array([ True,  True,  True, False, False,  True,  True,  True,  True,
        False]),
 array([False, False, False,  True,  True, False, False, False, False,
         True]))

In [22]:
pic_name="picture.jpg"
img = cv.imread(pic_name)
print(img.shape)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
out = last_matrix(img)


(328, 529, 3)


In [21]:
def last_matrix(image):
    return anchorcreater(pixelmatrix(rgb_to_hex_image(image)))

In [23]:
out.shape

(10, 10, 33, 53, 24)